# Test Plugin Integration

> Test the Voxtral VLLM plugin with the transcription plugin system

In [1]:
import logging
import json
import numpy as np
from pathlib import Path

# Import PluginManager from the generic plugin system
from cjm_plugin_system.core.manager import PluginManager
from cjm_transcription_plugin_system.plugin_interface import TranscriptionPlugin
from cjm_transcription_plugin_system.core import AudioData
from cjm_transcription_plugin_voxtral_vllm.plugin import VoxtralVLLMPlugin

## Test Direct Plugin Usage

In [2]:
# Create plugin directly
plugin = VoxtralVLLMPlugin()

# Check basic properties
print(f"Plugin: {plugin.name} v{plugin.version}")
print(f"Available: {plugin.is_available()}")
print(f"Supported formats: {', '.join(plugin.supported_formats)}")
print(f"Supports streaming: {plugin.supports_streaming()}")

Plugin: voxtral_vllm v1.0.0
Available: True
Supported formats: wav, mp3, flac, m4a, ogg, webm, mp4, avi, mov
Supports streaming: True


In [3]:
# Get configuration schema
schema = plugin.get_config_schema()
print("Configuration options:")
print(f"- Required: {schema.get('required', [])}")
print(f"- Total properties: {len(schema['properties'])}")
print("\nAvailable models:")
for model in schema['properties']['model_id']['enum']:
    print(f"  - {model}")
print("\nServer modes:")
for mode in schema['properties']['server_mode']['enum']:
    print(f"  - {mode}: {schema['properties']['server_mode']['description']}" if mode == "managed" else f"  - {mode}")

Configuration options:
- Required: ['model_id']
- Total properties: 14

Available models:
  - mistralai/Voxtral-Mini-3B-2507
  - mistralai/Voxtral-Small-24B-2507

Server modes:
  - managed: 'managed': plugin manages server lifecycle, 'external': connect to existing server
  - external


## Test with Plugin Manager

In [4]:
# Setup logging
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

# Create plugin manager
manager = PluginManager(plugin_interface=TranscriptionPlugin)

In [5]:
# Option 1: Load plugin from module directly (for development)
# This works even without the package being installed
import sys
import os

# Add parent directory to path to import the plugin module
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Import and register the plugin
from cjm_transcription_plugin_voxtral_vllm.plugin import VoxtralVLLMPlugin

# Create a temporary module file for the plugin manager to load
temp_plugin_file = Path("temp_voxtral_vllm_plugin.py")
with open(temp_plugin_file, "w") as f:
    f.write("from cjm_transcription_plugin_voxtral_vllm.plugin import VoxtralVLLMPlugin\n")

# Load the plugin with managed server mode for testing
success = manager.load_plugin_from_module(
    str(temp_plugin_file),
    config={
        "model_id": "mistralai/Voxtral-Mini-3B-2507",
        "server_mode": "managed",
        "server_url": "http://localhost:8000"
    }
)

print(f"Plugin loaded: {success}")

# Clean up temp file
temp_plugin_file.unlink()

cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Initialized Voxtral VLLM plugin with model 'mistralai/Voxtral-Mini-3B-2507' in managed mode
cjm_plugin_system.core.manager.PluginManager - INFO - Loaded plugin from module: voxtral_vllm


Plugin loaded: True


In [6]:
# List loaded plugins
print("Loaded plugins:")
for meta in manager.list_plugins():
    print(f"  - {meta.name} v{meta.version} (enabled: {meta.enabled})")

Loaded plugins:
  - voxtral_vllm v1.0.0 (enabled: True)


In [7]:
# Get plugin configuration
current_config = manager.get_plugin_config("voxtral_vllm")
print("Current Voxtral VLLM configuration:")
config_subset = {k: current_config[k] for k in ["model_id", "server_mode", "server_url", "language"] if k in current_config}
print(json.dumps(config_subset, indent=2))

Current Voxtral VLLM configuration:
{
  "model_id": "mistralai/Voxtral-Mini-3B-2507",
  "server_mode": "managed",
  "server_url": "http://localhost:8000",
  "language": "en"
}


## Test Configuration Management

In [8]:
# Validate different configurations
test_configs = [
    ({"model_id": "mistralai/Voxtral-Small-24B-2507"}, "Valid: switching to Small model"),
    ({"model_id": "invalid_model"}, "Invalid: bad model name"),
    ({"temperature": 0.5}, "Valid: adjusting temperature"),
    ({"server_port": 9000}, "Valid: changing server port"),
    ({"gpu_memory_utilization": 1.5}, "Invalid: GPU memory out of range"),
]

for config, description in test_configs:
    is_valid, error = manager.validate_plugin_config("voxtral_vllm", config)
    print(f"{description}")
    print(f"  Config: {config}")
    print(f"  Valid: {is_valid}")
    if error:
        print(f"  Error: {error[:100]}...")
    print()

Valid: switching to Small model
  Config: {'model_id': 'mistralai/Voxtral-Small-24B-2507'}
  Valid: True

Invalid: bad model name
  Config: {'model_id': 'invalid_model'}
  Valid: False
  Error: 'invalid_model' is not one of ['mistralai/Voxtral-Mini-3B-2507', 'mistralai/Voxtral-Small-24B-2507']...

Valid: adjusting temperature
  Config: {'temperature': 0.5}
  Valid: False
  Error: 'model_id' is a required property

Failed validating 'required' in schema:
    {'$schema': 'http://j...

Valid: changing server port
  Config: {'server_port': 9000}
  Valid: False
  Error: 'model_id' is a required property

Failed validating 'required' in schema:
    {'$schema': 'http://j...

Invalid: GPU memory out of range
  Config: {'gpu_memory_utilization': 1.5}
  Valid: False
  Error: 'model_id' is a required property

Failed validating 'required' in schema:
    {'$schema': 'http://j...



In [9]:
# Update configuration
new_config = {
    "temperature": 0.0,
    "language": "en",
    "streaming": True,
}

success = manager.update_plugin_config("voxtral_vllm", new_config, merge=True)
print(f"Configuration updated: {success}")

if success:
    updated_config = manager.get_plugin_config("voxtral_vllm")
    print("\nUpdated configuration:")
    for key in new_config:
        print(f"  {key}: {updated_config[key]}")

cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Cleaning up Voxtral VLLM plugin
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Cleanup completed successfully
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Initialized Voxtral VLLM plugin with model 'mistralai/Voxtral-Mini-3B-2507' in managed mode
cjm_plugin_system.core.manager.PluginManager - INFO - Updated configuration for plugin: voxtral_vllm


Configuration updated: True

Updated configuration:
  temperature: 0.0
  language: en
  streaming: True


## Test with Managed Server Mode

This section demonstrates using the plugin with a managed vLLM server that the plugin starts and stops automatically.

In [10]:
# Create a new manager for managed server testing
manager_managed = PluginManager(plugin_interface=TranscriptionPlugin)

# Create temp module file
temp_plugin_file = Path("temp_voxtral_vllm_managed.py")
with open(temp_plugin_file, "w") as f:
    f.write("from cjm_transcription_plugin_voxtral_vllm.plugin import VoxtralVLLMPlugin\n")

# Load plugin with managed server mode
success = manager_managed.load_plugin_from_module(
    str(temp_plugin_file),
    config={
        "model_id": "mistralai/Voxtral-Mini-3B-2507",
        "server_mode": "managed",
        "server_port": 8001,  # Use different port to avoid conflicts
        "auto_start_server": True,
        "gpu_memory_utilization": 0.85,
        "capture_server_logs": True,  # Enable log capture to see server startup progress
    }
)

print(f"Managed plugin loaded: {success}")
temp_plugin_file.unlink()

cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Initialized Voxtral VLLM plugin with model 'mistralai/Voxtral-Mini-3B-2507' in managed mode
cjm_plugin_system.core.manager.PluginManager - INFO - Loaded plugin from module: voxtral_vllm


Managed plugin loaded: True


In [11]:
from nbdev.config import get_config
from pathlib import Path

config = get_config()
project_dir = config.config_path
test_dir = project_dir/"./test_files/"
audio_path = test_dir/"short_test_audio.mp3"
assert audio_path.exists(), f"Test audio file not found at {audio_path}"

## Test with Managed Server (Auto-start)

This test will automatically start a vLLM server if you have vLLM installed and a GPU available.

In [12]:
# Test transcription with managed server (will auto-start server)
try:
    print("Testing with managed server (this may take a minute to start the server)...")
    result = manager_managed.execute_plugin("voxtral_vllm", audio_path)
    print("\nTranscription result:")
    print(f"  Text: {result.text}")
    print(f"  Server mode: {result.metadata.get('server_mode')}")
except Exception as e:
    print(f"Error: {e}")
    print("This requires vLLM to be installed and a GPU to be available")

cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Starting vLLM server...


Testing with managed server (this may take a minute to start the server)...


SERVER IS NOT RUNNING


Starting vLLM server with model mistralai/Voxtral-Mini-3B-2507...
10-21 17:42:45 INFO 10-21 17:42:45 [__init__.py:241] Automatically detected platform cuda.

  🔍 Detecting platform...
10-21 17:42:46 (APIServer pid=721874) INFO 10-21 17:42:46 [api_server.py:1805] vLLM API server version 0.10.1.1
10-21 17:42:46 (APIServer pid=721874) INFO 10-21 17:42:46 [utils.py:326] non-default args: {'host': '0.0.0.0', 'port': 8001, 'model': 'mistralai/Voxtral-Mini-3B-2507', 'tokenizer_mode': 'mistral', 'max_model_len': 32768, 'config_format': 'mistral', 'load_format': 'mistral', 'gpu_memory_utilization': 0.85}
10-21 17:42:49 (APIServer pid=721874) INFO 10-21 17:42:49 [__init__.py:711] Resolved architecture: VoxtralForConditionalGeneration
10-21 17:42:50 (APIServer pid=721874) INFO 10-21 17:42:50 [__init__.py:2816] Downcasting torch.float32 to torch.bfloat16.
10-21 17:42:50 (APIServer pid=721874) INFO

cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Processing audio with Voxtral mistralai/Voxtral-Mini-3B-2507 via vLLM


10-21 17:43:19 (APIServer pid=721874) INFO:     127.0.0.1:47126 - "GET /health HTTP/1.1" 200 OK
✅ vLLM server is ready at http://localhost:8001

10-21 17:43:21 (APIServer pid=721874) WARNING 10-21 17:43:21 [registry.py:183] VoxtralProcessorAdapter did not return `BatchFeature`. Make sure to match the behaviour of `ProcessorMixin` when implementing custom processors.


httpx - INFO - HTTP Request: POST http://localhost:8001/v1/audio/transcriptions "HTTP/1.1 200 OK"
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Transcription completed: 43 words


10-21 17:43:21 (APIServer pid=721874) INFO:     127.0.0.1:49666 - "POST /v1/audio/transcriptions HTTP/1.1" 200 OK

Transcription result:
  Text: November the 10th, Wednesday, 9 p.m. I'm standing in a dark alley. After waiting several hours, the time has come. A woman with long, dark hair approaches. I have to act and fast before she realizes what has happened. I must find out.
  Server mode: managed
10-21 17:43:21 (APIServer pid=721874) INFO:     127.0.0.1:49670 - "GET /health HTTP/1.1" 200 OK
10-21 17:43:22 (APIServer pid=721874) INFO:     127.0.0.1:49666 - "POST /v1/audio/transcriptions HTTP/1.1" 200 OK
10-21 17:43:22 (APIServer pid=721874) INFO:     127.0.0.1:49686 - "GET /health HTTP/1.1" 200 OK


## Test Streaming Support

In [13]:
# Test streaming transcription
try:
    print("Testing streaming transcription:")
    print("Streaming output: ", end="")
    for chunk in manager_managed.execute_plugin_stream("voxtral_vllm", audio_path):
        print(chunk, end="", flush=True)
    print("\n\nStreaming completed!")
except Exception as e:
    print(f"\nError: {e}")
    print("Make sure a vLLM server is running on port 8000")

cjm_plugin_system.core.manager.PluginManager - INFO - Using streaming mode for plugin voxtral_vllm
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Streaming transcription with Voxtral mistralai/Voxtral-Mini-3B-2507 via vLLM


Testing streaming transcription:
Streaming output: 

httpx - INFO - HTTP Request: POST http://localhost:8001/v1/audio/transcriptions "HTTP/1.1 200 OK"


November the 10th, Wednesday, 9 p.m. I'm standing in a dark alley. After waiting several hours, the time has come. A woman with long, dark hair approaches. I have to act and fast before she realizes what has happened. I must find out.

Streaming completed!


## Test Plugin Lifecycle

In [14]:
# Test disabling and enabling
print("Testing plugin lifecycle:")

# Disable plugin
manager.disable_plugin("voxtral_vllm")
print(f"Plugin disabled: {not manager.plugins['voxtral_vllm'].enabled}")

# Try to execute while disabled (should fail)
try:
    manager.execute_plugin("voxtral_vllm", audio_path)
except ValueError as e:
    print(f"Expected error: {e}")

# Re-enable plugin
manager.enable_plugin("voxtral_vllm")
print(f"Plugin re-enabled: {manager.plugins['voxtral_vllm'].enabled}")

Testing plugin lifecycle:
Plugin disabled: True
Expected error: Plugin voxtral_vllm is disabled
Plugin re-enabled: True


In [15]:
# Clean up
print("\nCleaning up...")

# Unload managed server plugin
manager.unload_plugin("voxtral_vllm")
print(f"Managed server plugin unloaded. Plugins remaining: {len(manager.list_plugins())}")

# Unload managed server plugin (will stop the server if running)
if 'voxtral_vllm' in manager_managed.plugins:
    manager_managed.unload_plugin("voxtral_vllm")
    print(f"Managed server plugin unloaded. Plugins remaining: {len(manager_managed.list_plugins())}")

cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Cleaning up Voxtral VLLM plugin
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Cleanup completed successfully
cjm_plugin_system.core.manager.PluginManager - INFO - Unloaded plugin: voxtral_vllm
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Cleaning up Voxtral VLLM plugin
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Stopping managed vLLM server



Cleaning up...
Managed server plugin unloaded. Plugins remaining: 0
Stopping vLLM server...


cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Cleanup completed successfully
cjm_plugin_system.core.manager.PluginManager - INFO - Unloaded plugin: voxtral_vllm


Server stopped
Managed server plugin unloaded. Plugins remaining: 0


## Test Entry Point Discovery (After Installation)

This will work after the package is installed with `pip install -e .` or `pip install cjm-transcription-plugin-voxtral-vllm`

In [16]:
# This will only work after the package is installed
print("Testing entry point discovery:")
manager2 = PluginManager(plugin_interface=TranscriptionPlugin)

# Discover plugins via entry points
discovered = manager2.discover_plugins()
print(f"\nDiscovered {len(discovered)} plugin(s) via entry points:")
for plugin_meta in discovered:
    print(f"  - {plugin_meta.name} v{plugin_meta.version} from {plugin_meta.package_name}")

# Load discovered plugins
for plugin_meta in discovered:
    if plugin_meta.name == "voxtral_vllm":
        success = manager2.load_plugin(
            plugin_meta,
            config={
                "model_id": "mistralai/Voxtral-Mini-3B-2507",
                "server_mode": "managed",
                "server_url": "http://localhost:8000"
            }
        )
        print(f"\nLoaded {plugin_meta.name}: {success}")

cjm_plugin_system.core.manager.PluginManager - INFO - Discovered plugin: voxtral_vllm v0.0.2 from package cjm-transcription-plugin-voxtral-vllm
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Initialized Voxtral VLLM plugin with model 'mistralai/Voxtral-Mini-3B-2507' in managed mode
cjm_plugin_system.core.manager.PluginManager - INFO - Loaded plugin: voxtral_vllm


Testing entry point discovery:

Discovered 1 plugin(s) via entry points:
  - voxtral_vllm v0.0.2 from cjm-transcription-plugin-voxtral-vllm

Loaded voxtral_vllm: True


In [17]:
# Test transcription with discovered plugin
if len(discovered) > 0 and "voxtral_vllm" in [p.name for p in discovered]:
    try:
        print(f"Transcribing: {audio_path}")
        result = manager2.execute_plugin("voxtral_vllm", audio_path)
        print("Transcription result:")
        print(f"  Text: {result.text}")
        print(f"  Metadata: {result.metadata}")
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure a vLLM server is running on port 8000")
    
    # Clean up
    manager2.get_plugin('voxtral_vllm').cleanup()

cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Starting vLLM server...


Transcribing: /mnt/SN850X_8TB_EXT4/Projects/GitHub/cj-mills/cjm-transcription-plugin-voxtral-vllm/test_files/short_test_audio.mp3


SERVER IS NOT RUNNING


Starting vLLM server with model mistralai/Voxtral-Mini-3B-2507...
10-21 17:43:26 INFO 10-21 17:43:26 [__init__.py:241] Automatically detected platform cuda.

  🔍 Detecting platform...
10-21 17:43:27 (APIServer pid=723725) INFO 10-21 17:43:27 [api_server.py:1805] vLLM API server version 0.10.1.1
10-21 17:43:27 (APIServer pid=723725) INFO 10-21 17:43:27 [utils.py:326] non-default args: {'host': '0.0.0.0', 'model': 'mistralai/Voxtral-Mini-3B-2507', 'tokenizer_mode': 'mistral', 'max_model_len': 32768, 'config_format': 'mistral', 'load_format': 'mistral', 'gpu_memory_utilization': 0.85}
10-21 17:43:30 (APIServer pid=723725) INFO 10-21 17:43:30 [__init__.py:711] Resolved architecture: VoxtralForConditionalGeneration
10-21 17:43:31 (APIServer pid=723725) INFO 10-21 17:43:31 [__init__.py:2816] Downcasting torch.float32 to torch.bfloat16.
10

cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Processing audio with Voxtral mistralai/Voxtral-Mini-3B-2507 via vLLM


10-21 17:43:53 (APIServer pid=723725) INFO:     127.0.0.1:33700 - "GET /health HTTP/1.1" 200 OK
✅ vLLM server is ready at http://localhost:8000

10-21 17:43:53 (APIServer pid=723725) WARNING 10-21 17:43:53 [registry.py:183] VoxtralProcessorAdapter did not return `BatchFeature`. Make sure to match the behaviour of `ProcessorMixin` when implementing custom processors.


httpx - INFO - HTTP Request: POST http://localhost:8000/v1/audio/transcriptions "HTTP/1.1 200 OK"
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Transcription completed: 43 words
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Cleaning up Voxtral VLLM plugin
cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Stopping managed vLLM server


10-21 17:43:54 (APIServer pid=723725) INFO:     127.0.0.1:33714 - "POST /v1/audio/transcriptions HTTP/1.1" 200 OK
Transcription result:
  Text: November the 10th, Wednesday, 9 p.m. I'm standing in a dark alley. After waiting several hours, the time has come. A woman with long, dark hair approaches. I have to act and fast before she realizes what has happened. I must find out.
  Metadata: {'model': 'mistralai/Voxtral-Mini-3B-2507', 'language': 'en', 'server_mode': 'managed', 'temperature': 0.0}
10-21 17:43:54 (APIServer pid=723725) INFO:     127.0.0.1:33730 - "GET /health HTTP/1.1" 200 OK
Stopping vLLM server...


cjm_transcription_plugin_voxtral_vllm.plugin.VoxtralVLLMPlugin - INFO - Cleanup completed successfully


Server stopped
